In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/'NLP Hate Speech'/

/content/drive/MyDrive/NLP Hate Speech


In [1]:
# Import the libraries
import csv
import stanza      
# download the stanza library for Hindi NLP
stanza.download('hi', processors='tokenize,pos,lemma')          

2022-11-15 19:17:23 INFO: Downloading these customized packages for language: hi (Hindi)...
| Processor       | Package |
-----------------------------
| tokenize        | hdtb    |
| pos             | hdtb    |
| lemma           | hdtb    |
| pretrain        | hdtb    |
| backward_charlm | oscar   |
| forward_charlm  | oscar   |

2022-11-15 19:17:23 INFO: File exists: C:\Users\shubham gupta\stanza_resources\hi\tokenize\hdtb.pt
2022-11-15 19:17:24 INFO: File exists: C:\Users\shubham gupta\stanza_resources\hi\pos\hdtb.pt
2022-11-15 19:17:24 INFO: File exists: C:\Users\shubham gupta\stanza_resources\hi\lemma\hdtb.pt
2022-11-15 19:17:26 INFO: File exists: C:\Users\shubham gupta\stanza_resources\hi\pretrain\hdtb.pt
2022-11-15 19:17:26 INFO: File exists: C:\Users\shubham gupta\stanza_resources\hi\backward_charlm\oscar.pt
2022-11-15 19:17:27 INFO: File exists: C:\Users\shubham gupta\stanza_resources\hi\forward_charlm\oscar.pt
2022-11-15 19:17:27 INFO: Finished downloading models and saved to

# Subjectivity Analysis
- We use Sentiment lexicon resource for hindi called Hindi Sentiwordnet.
- It has around 3000 prior-polarity subjective clues with POS tag, positive score, negative score and related terms(separated by comma)

In [152]:
SUBJCLUE = []

# Write the data from the file into the list
with open('SUBJCLUE.txt', encoding="utf8") as f:
	for line in f:                
		x = line.split()            
		x[4] = x[4].split(',')      
		SUBJCLUE.append(x)          

# After this, the data would be in this form:
# ['POS tag', 'SYSNET ID(Hindi WN)', 'Positive score', 'Negative score', List of related words]

# Printing the data
for key in SUBJCLUE[:5]:
  	print(key, key[4])

['a', '10363', '0.0', '0.0', ['अनौपचारिक']] ['अनौपचारिक']
['a', '2627', '0.0', '0.75', ['मृत']] ['मृत']
['a', '11476', '0.125', '0.0', ['परवर्ती']] ['परवर्ती']
['a', '28106', '0.25', '0.375', ['अच्छा', 'बढ़िया']] ['अच्छा', 'बढ़िया']
['a', '1156', '0.875', '0.0', ['सौभाग्यशाली', 'खुशकिस्मत', 'खुशनसीब', 'तक़दीर_वाला', 'नसीब_वाला', 'भाग्यवान', 'भाग्यशाली', 'ख़ुशक़िस्मत', 'ख़ुशनसीब']] ['सौभाग्यशाली', 'खुशकिस्मत', 'खुशनसीब', 'तक़दीर_वाला', 'नसीब_वाला', 'भाग्यवान', 'भाग्यशाली', 'ख़ुशक़िस्मत', 'ख़ुशनसीब']


## Reading the data


Note: The Dataset is a csv file with Fields corresponding to Unique ID, Post, Labels Set 

In [153]:
def get_data(filename, rows):                     # function to get data from csv file
    with open(filename, 'r', encoding="utf8") as csvfile:        # opening csv file
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        
        # extracting field names through first row
        fields = next(csvreader)
    
        # extracting each data row one by one
        for row in csvreader:
            rows.append(row)
    
        # get total number of rows
        print("Total no. of rows: %d"%(csvreader.line_num))
    return rows

In [154]:
rows = []
rows = get_data("dataset.csv", rows)

# printing first 5 rows
for row in rows[:5]:
    # parsing each column of a row
    for col in row:
        print("%10s"%col),
    print('\n')

Total no. of rows: 30728


         1
दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।

उनका निधन हम सब के लिए अपूरणीय क्षति है।
ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।
ऊं शांति!!!
non-hostile




         2
भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी  करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो
defamation






In [155]:
tot = 0
for row in rows:
    row.append(tot)

## Checking score

Finding positive, negative and total scores for each sentence



In [156]:
while [0] in rows:
    rows.remove([0])

In [157]:
len(rows)

8192

In [158]:
for i in rows:
    print(i)
    break

['1', 'दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!', 'non-hostile', 0]


In [159]:
count = 0
for key in SUBJCLUE:
    subjlist = key[4]

    # subjlist = ['इच्छा', 'आत्मा', 'इतने']
    for row in rows:
        if any([subjword in row[1] for subjword in subjlist]):
            count += 1

            # Get the score for the word
            pos = float(key[2])
            neg = float(key[3])
            tot = pos - neg
            
            row[3] += tot

# printing the number of occurences of sentiment words in dataset
print(count)            

105374


# Build the Hate Lexicon


In [160]:
SYNSET = []                                                     # SYNSET is a list of lists
with open('Synset.txt', encoding= 'unicode_escape') as f:       # opening synset.txt file
    for line in f:                                              # iterating through SYNSET        
        x = line.split()                                        # splitting lines
        x[3] = x[3].split(':')                                  # [3] is the synonyms
        SYNSET.append(x)                                        # append to SYNSET

In [161]:
# dataset is a string
dataset = "".join(row[1]  for row in rows)                                                 

In [162]:
# nlp is a pipeline for processing text
nlp = stanza.Pipeline('hi',processors='tokenize,pos,lemma')     

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [163]:
# doc is a document object
doc = nlp(dataset)                                              

In [164]:
verbs_content = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'VERB']

In [165]:
strongly_negative_words = []
weakly_negative_words = []
for line in SUBJCLUE:
    totalscore = float(line[2]) - float(line[3])
    if totalscore < -0.25 :
        strongly_negative_words += [word for word in line[4]]
    elif totalscore < 0:
        weakly_negative_words += [word for word in line[4]]

In [166]:
def Getsynset(word):
    syn = [word]
    flag = False
    for line in SYNSET:
        if line[1] == "03":
            for verb in line[3]:
                if word == verb:
                    flag = True
                    break
            if flag:
                syn = line[3]
                break
    return syn

In [167]:
slist = ["लड़ना" , "मारना" , "लूटना" , "पीटना" , "कूटना" , "भेदभाव" ,"फोड़ना", "तोड़ना", "उखाड़ना" ]    # slist is a list of verbs 
hlex = slist.copy()     
for word in slist:
    hlex += [v for v in Getsynset(word) if v in verbs_content]

In [168]:
# open themenouns.txt in read
themed_nouns = open('themenouns.txt','r', encoding="utf8")

# list of theme nouns
themenouns = [line.rstrip('\n') for line in themed_nouns]

# printing the theme nouns list
print(themenouns)                               

['बीजेपी ', 'मोदी ', 'माओवादियों ', 'इस्लाम ', 'धमकी ', 'सुरक्षा ', 'धर्म ', 'साले ', 'कुत्ते ', 'कुतिया', 'कुते ', 'कुत्ती', 'कुत्तो', 'कमीना', 'कमीनी', 'साला', 'साली', 'हरामी', 'हरामखोर', 'बहनचोद', 'मादरचोद', 'चूतिया', 'चूत', 'चुत', 'टट्टी', 'नाजायज', 'झांट', 'सुअर', 'बेटीचोद', 'गांड', 'भोसड़ी', 'रन्डी', 'रांड', 'भड़वे', 'लौड़ा', 'लोडे', 'लवड़ा', 'चोर ', 'औलाद ', 'चीन ', 'औकात ', 'चुनौती', 'कश्मीर ', 'ज़ुल्म ', 'मरकज ', 'भारत', 'आतंकवाद', 'इस्लामिक', 'तालिबानी', 'हिन्दू ', 'अर्नब ', 'गद्दारों ', 'कलंकित ', 'तोड़फोड़ ', 'शिवसेना ', 'मंदिर ', 'राम ', 'हिन्दुओं ', 'शूद्र ', 'मुसलमान ', 'विपक्षी ', 'आग ', 'कॉंग्रेस ', 'आतंकवादी ', 'डायन ', 'पलटू ', 'फेंकूँ ', 'पाकिस्तान ', 'जिंदाबाद ', 'आतंकी ', 'आतंकी ', 'आतंकियों ', 'हिंदुस्तान ', 'हिन्दुओं', 'नेता', 'गुलाम ', 'पीओके ', 'आरएसएस ', 'भैंसियो ', 'चमचों ', 'पिल्ला ', 'गधे ', 'तबाह ', 'मुसलमान ', 'मुसलमानों ', 'मौलवी ', 'धर्म ']


# Hate speech Detection Algorithm

In [169]:
print(strongly_negative_words)  # printing the strongly negative words
print(weakly_negative_words)    # printing the weakly negative words
print(hlex)                     # printing the hlex words
print(themenouns)               # printing the themenouns 

['मृत', 'दुर्भाग्यशाली', 'अभागा', 'बदनसीब', 'भाग्यहीन', 'मनहूस', 'बदकिस्मत', 'मंदभाग्य', 'बदक़िस्मत', 'दईमारा', 'कमबख्त', 'कमबख़्त', 'अधन्य', 'अभागी', 'आवासहीन', 'आश्रयहीन', 'गृहहीन', 'गृहविहीन', 'बेघर', 'बेघरबार', 'अगतिक', 'अगेह', 'अनिकेत', 'बदबूदार', 'दुर्गंधपूर्ण', 'दुर्गंधयुक्त', 'दुर्गंधित', 'ढीला', 'अश्लिष्ट', 'असंयुक्त', 'असंयोजित', 'असंबद्ध', 'अलग', 'अजुड़ा', 'अजोड़', 'पृथक्', 'जुदा', 'पृथक', 'अपृक्त', 'पराधीन', 'गुलाम', 'परतंत्र', 'अन्याधीन', 'अपरवश', 'परवश', 'अवश', 'अबस', 'ढीला', 'जड़', 'अचैतन्य', 'जड़त्वयुक्त', 'स्थूल', 'अजैव', 'भौतिक', 'अचेतन', 'चेतनारहित', 'अजीव', 'अनात्म', 'आत्मारहित', 'अफल', 'अफलित', 'फलहीन', 'फलरहित', 'फलविहीन', 'निस्संतान', 'निःसंतान', 'बेऔलाद', 'संतानहीन', 'संतानरहित', 'अऊत', 'अनपत्य', 'भली-भाँति', 'भली_भाँति', 'भरपूर', 'भलीभाँति', 'भली-भांति', 'भली_भांति', 'भलीभांति', 'पंखहीन', 'पक्षरहित', 'अपक्ष', 'अपच्छी', 'अपत्र', 'अतकनीकी', 'ग़ैरतकनीकी', 'गैरतकनीकी', 'तकनीकहीन', 'प्राविधिहीन', 'दुर्गुणी', 'अगुणी', 'अपगुणी', 'ऐबी', 'खोटा', 'विपरीत', 'प्रतिकूल', 'ख

In [170]:
for row in rows:
    row.append("")               # appending "" to row to make the size uniform
    row.append("")               # appending "" to row to make the size uniform

In [171]:
for row in rows:
    print(row)                # printing the text of the row
    break

['1', 'दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!', 'non-hostile', -0.375, '', '']


In [172]:
def feature_selection(rows, idx, strong = False, hlex_flag = False, weak = False, theme = False, subjectivity = False):
	for row in rows:                                                      # Iterate over all rows
		if subjectivity and not (row[3] <= -0.5 or row[3] >= 0.5):
			continue				

		strongcount = 0                                                   # Initialize strong count
		hlexcount = 0                                                     # Initialize hlex count
		weakcount = 0                                                     # Initialize weak count
		themecount = 0                                                    # Initialize theme count     

		if strong and any([word in row[1] for word in strongly_negative_words]):     # If any of the strongly negative words are in the tweet
			strongcount += 1                                                # Increment strong count
		if hlex_flag and any([word in row[1] for word in hlex]):                        # If any of the hlex words are in the tweet
			hlexcount += 1                                                  # Increment hlex count
		if weak and any([word in row[1] for word in weakly_negative_words]):       # If any of the weakly negative words are in the tweet
			weakcount += 1                                                  # Increment weak count    
		if theme and any([word in row[1] for word in themenouns]):                  # If any of the theme nouns are in the tweet
			themecount += 1                                                 # Increment theme count

		if strongcount >= 2:                                              # If strong count is greater than or equal to 2
			row[idx] = "strongly hateful"                                # Append strongly hateful to the row 
		elif strongcount == 1:                                            # Else if strong count is 1
			row[idx] = "strongly hateful" if hlexcount >= 1 or themecount >= 1 else "weakly hateful"                      # If hlex count is 1 or theme count is 1
		elif strongcount == 0:                                            # Else if strong count is 0
			if themecount >= 1 and hlexcount >= 1:                          # If theme count is 1 and hlex count is 1
				row[idx] = "strongly hateful"                                # Append strongly hateful to the row
			elif (themecount >=1 and weakcount >= 1) or hlexcount == 1:                         # Else if theme count is 1 and weak count is 1
				row[idx] = "weakly hateful"                                  # Append weakly hateful to the row
			else:                                                           # Else
				row[idx] = "No Hate"                                        # Append No Hate to the row

In [173]:
def score_calculation(rows, idx):

    # total rows = toal number of rows
    total_rows = [row for row in rows]

    # no_hate_rows = number of rows that are marked to have no hate
    no_hate_rows = [row for row in rows if row[idx] == "No Hate"]

    # correct_no_hate_rows = number of rows that have no hate speech and are correctly marked
    correct_no_hate_rows = [row for row in no_hate_rows if row[idx] == "No Hate" and row[2] == "non-hostile"]

    # weak_hate_rows = number of rows that are marked to have weak hate
    weak_hate_rows = [row for row in rows if row[idx] == "weakly hateful"]

    # correct_weak_hate_rows = number of rows that have weak hate speech and are correctly marked
    correct_weak_hate_rows = [row for row in weak_hate_rows if row[idx] == "weakly hateful" and (row[2] == "fake" or row[2] == "defamation")]

    # strong_hate_rows = number of rows that are marked to have strong hate
    strong_hate_rows = [row for row in rows if row[idx] == "strongly hateful"]

    # correct_strong_hate_rows = number of rows that have strong hate speech and are correctly marked
    correct_strong_hate_rows = [row for row in strong_hate_rows if row[idx] == "strongly hateful" and row[2] != "non-hostile" and row[2] != "fake" and row[2] != "defamation"]

    # false negatives in the no hate list
    false_neg_no_hate = [row for row in no_hate_rows if row[2] == "non-hostile" and row[idx] != "No Hate"]

    # false negatives in the weak hate list
    false_neg_weak_hate = [row for row in weak_hate_rows if row[2] == "fake" or row[2] == "defamation" and row[idx] != "weakly hateful"]
    
    # false negatives in the strong hate list
    false_neg_strong_hate = [row for row in strong_hate_rows if row[2] != "non-hostile" and row[2] != "fake" and row[2] != "defamation" and row[idx] != "strongly hateful"]

    # calculating precision
    precision = (len(correct_no_hate_rows)+len(correct_strong_hate_rows)+len(correct_weak_hate_rows))/(len(no_hate_rows)+len(strong_hate_rows)+len(weak_hate_rows))

    # calculating recall
    recall = (len(correct_no_hate_rows)+len(correct_strong_hate_rows)+len(correct_weak_hate_rows))/(len(correct_no_hate_rows)+len(correct_strong_hate_rows)+len(correct_weak_hate_rows)+len(false_neg_no_hate)+len(false_neg_strong_hate)+len(false_neg_weak_hate))

    # calculating F1 score
    f1 = 2*precision*recall/(precision+recall)

    print("Total no. of rows: {}".format(len(total_rows)))                  # total no. of rows
    print("No Hate: {}".format(len(no_hate_rows)))                          # no hate rows
    print("Actual no hate: {}".format(len(correct_no_hate_rows)))           # actual no hate rows
    print("Weak Hate: {}".format(len(weak_hate_rows)))                      # weak hate rows      
    print("Actual weak hate: {}".format(len(correct_weak_hate_rows)))       # actual weak hate rows
    print("Strong Hate: {}".format(len(strong_hate_rows)))                  # strong hate rows
    print("Actual strong hate: {}".format(len(correct_strong_hate_rows)))   # actual strong hate rows

    # print precision
    print("Precision: {}".format(precision))

    # print recall
    print("Recall: {}".format(recall))

    # print f1
    print("F-score: {}".format(f1))    


## Calculating Scores without Subjective Analysis

### Only Semantic feature set

In [174]:
feature_selection(rows, 4, strong = True, hlex_flag = False, weak = True, theme = False, subjectivity = False)
score_calculation(rows, 4)  

Total no. of rows: 8192
No Hate: 3558
Actual no hate: 2098
Weak Hate: 4634
Actual weak hate: 1128
Strong Hate: 0
Actual strong hate: 0
Precision: 0.393798828125
Recall: 0.7883675464320625
F-score: 0.5252360794529469


### Semantic + Hate Lexicon

In [175]:
feature_selection(rows, 4, strong = True, hlex_flag = True, weak = True, theme = False, subjectivity = False)
score_calculation(rows, 4)  

Total no. of rows: 8192
No Hate: 3544
Actual no hate: 2091
Weak Hate: 4627
Actual weak hate: 1125
Strong Hate: 21
Actual strong hate: 13
Precision: 0.3941650390625
Recall: 0.7885225885225885
F-score: 0.525596158541548


### Semantic + Hate Lexicon + Thematic Nouns

In [176]:
feature_selection(rows, 4, strong = True, hlex_flag = True, weak = True, theme = True, subjectivity = False)
score_calculation(rows, 4)  

Total no. of rows: 8192
No Hate: 2857
Actual no hate: 1829
Weak Hate: 3407
Actual weak hate: 808
Strong Hate: 1928
Actual strong hate: 800
Precision: 0.4195556640625
Recall: 0.8478046373951653
F-score: 0.5613261473134085


## Calculating Scores with Subjective Analysis

In [177]:
subj_rows = [row for row in rows if row[3] <= -0.5 or row[3] >= 0.5]

print("Number of Subjective Sentences: {}".format(len(subj_rows))) 

Number of Subjective Sentences: 4121


### Semantic feature set

In [178]:
feature_selection(rows, 5, strong = True, hlex_flag = False, weak = True, theme = False, subjectivity = True)
score_calculation(rows, 5)  

Total no. of rows: 8192
No Hate: 565
Actual no hate: 368
Weak Hate: 3556
Actual weak hate: 869
Strong Hate: 0
Actual strong hate: 0
Precision: 0.3001698616840573
Recall: 0.6513954713006845
F-score: 0.4109634551495017


### Semantic + Hate Lexicon 

In [179]:
feature_selection(rows, 5, strong = True, hlex_flag = True, weak = True, theme = False, subjectivity = True)
score_calculation(rows, 5)  

Total no. of rows: 8192
No Hate: 565
Actual no hate: 368
Weak Hate: 3542
Actual weak hate: 867
Strong Hate: 14
Actual strong hate: 9
Precision: 0.30186847852462995
Recall: 0.6526757607555089
F-score: 0.4128090260494442


### Semantic + Hate Lexicon + Thematic Nouns

In [180]:
feature_selection(rows, 5, strong = True, hlex_flag = True, weak = True, theme = True, subjectivity = True)
score_calculation(rows, 5)  

Total no. of rows: 8192
No Hate: 433
Actual no hate: 307
Weak Hate: 2230
Actual weak hate: 526
Strong Hate: 1458
Actual strong hate: 652
Precision: 0.3603494297500607
Recall: 0.7848837209302325
F-score: 0.49392981872609343


In [181]:
rows

[['1',
  'दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!',
  'non-hostile',
  -0.375,
  'weakly hateful',
  ''],
 ['2',
  'भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी  करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो',
  'defamation',
  0.0,
  'weakly hateful',
  ''],
 ['3',
  'कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\nhttps://t.co/9rlQowAsFh #Delhi @ArvindKejriwal  @rajnathsingh @AmitShah @DRDO_India @WHO @crpfindia @ITBP_official',
  'non-hostile',
  0.875,
  'No Hate',
  'No Hate'],
 ['4',
  'गवर्नर कॉन्फ्रेंस में PM मो

In [188]:
len(rows)

8192

## Building Classifier to classify the extent/degree of hate speech on a grade scale from 1-5

In [183]:
res = []
def classifier(rows):    
    for row in rows:
        strongcount = 0
        hlexcount = 0
        weakcount = 0
        themecount = 0
        val = 1
        if any([word in row[1] for word in strongly_negative_words]):   # If any of the strongly negative words are in the tweet
            strongcount += 1                                              # Add 1 to strongcount  
        if any([word in row[1] for word in hlex]):                      # If any of the hlex words are in the tweet
            hlexcount += 1                                                # Add 1 to hlexcount
        if any([word in row[1] for word in weakly_negative_words]):     # If any of the weakly negative words are in the tweet
            weakcount += 1                                                # Add 1 to weakcount                  
        if any([word in row[1] for word in themenouns]):                # If any of the themenouns words are in the tweet
            themecount += 1                                                # Add 1 to themecount
    
        if strongcount >= 2:                                              # If strong count is greater than or equal to 2                            # Append strongly hateful to the row 
            val = 5
        elif strongcount == 1:                                            # Else if strong count is 1
            val = 4 if hlexcount >= 1 or themecount >= 1 else 3                        # If hlex count is 1 or theme count is 1                               # Append strongly hateful to the row
        elif not strongcount:                                            # Else if strong count is 0
            if themecount >= 1 and hlexcount >= 1:                          # If theme count is 1 and hlex count is 1                              # Append strongly hateful to the row
                val = 4
            elif themecount >=1 and weakcount >= 1:                         # Else if theme count is 1 and weak count is 1                                  # Append weakly hateful to the row
                val = 3
            elif hlexcount == 1:
                val = 2
            else:
                val = 1        
        res.append(row + [val])
    return res

In [184]:
res = classifier(rows)

## Exporting results into CSV File

In [185]:
output = [(uid, post, hate, val) for uid, post, labels, score, hate, subhate, val in res]

In [186]:
output

[('1',
  'दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!',
  'weakly hateful',
  3),
 ('2',
  'भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी  करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो',
  'weakly hateful',
  3),
 ('3',
  'कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\nhttps://t.co/9rlQowAsFh #Delhi @ArvindKejriwal  @rajnathsingh @AmitShah @DRDO_India @WHO @crpfindia @ITBP_official',
  'No Hate',
  1),
 ('4',
  'गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\nhttps://t.co/ZvKgxk6dbd',
  'stro

In [187]:
fields = ['Unique ID', 'Post', 'Hate Label' ,'Classifying the degree/extent of the hate speech on a grade scale of 1-5']

with open("results_output.csv", 'w', encoding="utf8") as csvfile:          

    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(output)